In [ ]:
# adws# prompt: load drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the directory where the model is saved
load_directory = "/content/drive/Shareddrives/517 nlp project/Models/Llama-3.2-3B-Instruct"

# Load the model and tokenizer from the specified directory
tokenizer = AutoTokenizer.from_pretrained(load_directory)
model = AutoModelForCausalLM.from_pretrained(load_directory)

print("Model and tokenizer successfully loaded from Google Drive!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
load_directory = "/content/drive/Shareddrives/517 nlp project/Models/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(load_directory)
model = AutoModelForCausalLM.from_pretrained(load_directory).to("cuda")

# Ensure the tokenizer has a pad_token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))  # Resize model's embedding matrix if a new token is added

# Prompt the model to generate the longest possible response
prompt = (
    "debate relgiion and atheism to completition"
)

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("cuda")

# Measure inference time
start_time = time.time()

# Generate output with maximum length
output = model.generate(
    inputs.input_ids,
    max_length=2048,  # Maximum length for generation
    temperature=0.8,  # Controls randomness; lower is more deterministic
    top_p=0.9,        # Controls diversity
    repetition_penalty=1.1,  # Penalizes repetitive output
    do_sample=True,   # Enables sampling
    use_cache=True,   # Optimizes performance by caching computations
    pad_token_id=tokenizer.pad_token_id  # Explicitly set pad_token_id
)

# Measure the time taken
end_time = time.time()

# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Calculate metrics
elapsed_time = end_time - start_time
tokens_generated = len(generated_text.split())  # Approximate number of tokens
tokens_per_second = tokens_generated / elapsed_time

# Print the response
print("--- Generated Response ---")
print(generated_text)

# Print performance statistics
print("\n--- Performance Statistics ---")
print(f"Elapsed Time: {elapsed_time:.2f} seconds")
print(f"Tokens Generated: {tokens_generated}")
print(f"Tokens per Second: {tokens_per_second:.2f}")
print(f"Prompt Length (Tokens): {inputs.input_ids.shape[-1]}")
print(f"Generated Length (Tokens): {tokens_generated}")


In [ ]:
import re

def extract_equation(text):
    """Extracts the equation from the generated text using regular expressions."""
    match = re.search(r"[-+*/()]?\d+\s*[-+*/()]\s*\d+", text)  # Find simple arithmetic operations
    if match:
        return match.group(0)  # Return the matched equation
    else:
        return ""  # Return empty string if no equation is found

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [ ]:
import pandas as pd

# Load dataset
dataset_path = "/content/drive/Shareddrives/517 nlp project/data/SVAMP/data/mawps-asdiv-a_svamp_without_questions/dev.csv"
df = pd.read_csv(dataset_path)

# Define formatting function - now without showing the answer
def format_question(row):
    # Get the numbers from the Numbers column
    numbers = row['Numbers'].split()

    # Replace number0, number1, etc with actual values
    body = row['Body']
    question = row['Ques']

    for i, num in enumerate(numbers):
        body = body.replace(f'number{i}', num)
        question = question.replace(f'number{i}', num)

    return f"Problem: {body}\nQuestion: {question}"

# Create formatted question column with actual numbers
df['formatted_question'] = df.apply(format_question, axis=1)

# Display first 3 examples with clear formatting
print("Sample formatted questions:\n")
for idx, row in df.head(3).iterrows():
    print(f"Example {idx + 1}:")
    print(f"{row['formatted_question']}")
    print(f"Answer (not shown to model): {row['Answer']}")
    print("---\n")

In [ ]:
def generate_math_prompt(question):
    prompt = f"""Solve this math word problem step by step:

{question}

Let's solve this step by step:
1) First, understand what is being asked
2) Then, identify the important information
3) Solve step by step
4) Finally, state the answer in the format: [ANSWER]<number>[/ANSWER]

Show your work:"""
    return prompt

def extract_answer(response):
    """Extract the numerical answer from between [ANSWER] tags"""
    try:
        # Look for answer between tags
        import re
        pattern = r'\[ANSWER\](\d+\.?\d*)\[\/ANSWER\]'
        match = re.search(pattern, response)
        if match:
            return float(match.group(1))

        # Fallback: look for last number after "answer is" or similar phrases
        patterns = [
            r'answer is[^\d]*(\d+\.?\d*)',
            r'answer:[^\d]*(\d+\.?\d*)',
            r'equals[^\d]*(\d+\.?\d*)',
            r'=\s*(\d+\.?\d*)'
        ]

        for pattern in patterns:
            match = re.search(pattern, response.lower())
            if match:
                return float(match.group(1))

        # Final fallback: last number in response
        numbers = re.findall(r'\d+\.?\d*', response)
        if numbers:
            return float(numbers[-1])

        return None
    except:
        return None

In [ ]:
# Run inference on test set
predictions = []
actual = []
start_time = time.time()

# ANSI color codes
BLUE = '\033[94m'
GREEN = '\033[92m'
RED = '\033[91m'
YELLOW = '\033[93m'
CYAN = '\033[96m'
PURPLE = '\033[95m'  # New color for model response
ENDC = '\033[0m'

# Take first 10 examples for testing
total_questions = len(df.head(10))

for idx, row in df.head(10).iterrows():
    question = row['formatted_question']
    prompt = generate_math_prompt(question)

    # Generate response
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("cuda")
    output = model.generate(
        inputs.input_ids,
        max_length=512,
        temperature=0.1,  # Lower temperature for more focused answers
        do_sample=True,
        num_beams=3,      # Add beam search for better coherence
        top_p=0.9,        # Nucleus sampling
        repetition_penalty=1.2  # Increased penalty for repetition
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract numerical answer
    predicted = extract_answer(response)
    if predicted is not None:
        predictions.append(predicted)
        actual.append(float(row['Answer']))

        is_correct = abs(predicted - float(row['Answer'])) < 0.01
        correct_color = GREEN if is_correct else RED

        # Print progress and time estimation
        questions_done = len(predictions)
        questions_left = total_questions - questions_done
        elapsed_time = time.time() - start_time
        avg_time_per_question = elapsed_time / questions_done if questions_done > 0 else 0
        estimated_time_left = questions_left * avg_time_per_question

        print(f"\n{CYAN}Progress: {questions_done}/{total_questions} questions{ENDC}")
        print(f"{CYAN}Average time per question: {avg_time_per_question:.1f} seconds{ENDC}")
        print(f"{CYAN}Estimated time remaining: {estimated_time_left/60:.1f} minutes{ENDC}\n")

        # Print results
        print(f"\n{YELLOW}Question {idx}:{ENDC}")
        print(f"{BLUE}Prompt: {question}{ENDC}")
        print(f"{YELLOW}Model response:{ENDC}")
        print(f"{PURPLE}{response}{ENDC}")  # Color the entire response
        print(f"{correct_color}Correct: {is_correct}{ENDC}")

In [ ]:
# Calculate accuracy
correct = 0
print(f"Mean absolute error: {errors.mean():.2f}")
print(f"Median absolute error: {np.median(errors):.2f}")

NOTE: FOR THE SVAMP DATASET, there are multiple folders. there are three datasets, easy math, med math, and harder math. for two of the datasets, they have a question version and a no question version. i find they are identicle. they also have a combined (all three) dataset (there is one labeled questions and one labeled no questions). just use one of the combined datasets